In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = torch.bfloat16
load_in_4bit = True
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.12.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset
ds = load_dataset("kayrab/patient-doctor-qa-tr-167732")
train_raw = ds["train"]
val_raw   = ds["test"]
print(len(train_raw), len(val_raw))


README.md: 0.00B [00:00, ?B/s]

train_cleaned.csv:   0%|          | 0.00/113M [00:00<?, ?B/s]

train_cleaned_shuffled.csv:   0%|          | 0.00/113M [00:00<?, ?B/s]

train_original.csv:   0%|          | 0.00/121M [00:00<?, ?B/s]

test_cleaned.csv:   0%|          | 0.00/14.1M [00:00<?, ?B/s]

test_cleaned_shuffled.csv:   0%|          | 0.00/14.1M [00:00<?, ?B/s]

test_original.csv:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/503196 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60000 [00:00<?, ? examples/s]

503196 60000


In [ ]:
import re
import unicodedata

RE_URL   = re.compile(r"(https?://\S+|www\.\S+)", re.IGNORECASE)
RE_EMAIL = re.compile(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", re.IGNORECASE)

RE_MAIL_TOKEN = re.compile(r"(?i)\b(e-?posta|e\s*posta|eposta|e-?mail|email|mail)\b")

RE_CONTACT_LINE = re.compile(
    r"(?im)^.*(@|\b(e-?posta|e\s*posta|eposta|e-?mail|email|mail)\b).*$"
)

RE_PHONE = re.compile(
    r"\b(?:\+?\d{1,3}\s*)?(?:0\s*)?(?:\(?\d{3}\)?[\s\-.]?)\d{3}[\s\-.]?\d{2}[\s\-.]?\d{2}\b"
)
RE_PHONE_TR_LOOSE = re.compile(r"(?:\+?90\s*)?(?:0\s*)?5\s*\d(\s*\d){8,12}")

RE_PROMO_LINE = re.compile(
    r"(?im)^.*\b(randevu|iletişim|telefon|tel\.?|gsm|whatsapp|muayenehane|klinik|web\s*sitesi|siteye\s*gir|adresine\s*gir|bizi\s*ara)\b.*$"
)

RE_ANY_STAR = re.compile(r"\*{2,}")
RE_WS = re.compile(r"\s+")

RE_DOSAGE = re.compile(
    r"\b(\d+(\.\d+)?)\s*(mg|ml|mcg|µg|iu|ünite|tablet|kapsül|şurup)\b",
    re.IGNORECASE
)
RE_FREQ = re.compile(
    r"\b(\d+\s*x\s*\d+|[12]\s*x\s*[12]|günde\s*\d+|günlük\s*\d+)\b",
    re.IGNORECASE
)

RE_RX_Q = re.compile(
    r"\b(hangi\s*ilaç|ilaç\s*öner|antidepresan|antibiyotik|doz|mg|reçete|kaç\s*mg|2x1|1x1)\b",
    re.IGNORECASE
)

RE_GREETING_NAME = re.compile(r"(?im)^\s*(merhaba|selam)\s+[a-zçğıöşü]{2,25}\b[^\n]*", re.IGNORECASE)
RE_DOCTOR_NAME_INLINE = re.compile(
    r"(?i)\b(?:prof|doç|doc|uzm|op|yrd)\.?\s*(?:dr\.?)?\s*[A-ZÇĞİÖŞÜ][a-zçğıöşü]+(?:\s+[A-ZÇĞİÖŞÜ][a-zçğıöşü]+){0,2}\b"
)

def norm(x: str) -> str:
    x = "" if x is None else str(x)
    x = unicodedata.normalize("NFKC", x)
    x = x.replace("\r\n", "\n").replace("\r", "\n")
    return x

def clean_text(x: str, is_answer: bool = False) -> str:
    x = norm(x)

    x = RE_CONTACT_LINE.sub(" ", x)

    x = RE_PROMO_LINE.sub(" ", x)

    x = RE_URL.sub(" ", x)
    x = RE_EMAIL.sub(" ", x)
    x = RE_PHONE.sub(" ", x)
    x = RE_PHONE_TR_LOOSE.sub(" ", x)

    x = RE_MAIL_TOKEN.sub(" ", x)

    if is_answer:
        x = RE_GREETING_NAME.sub(" ", x)
        x = RE_DOCTOR_NAME_INLINE.sub(" ", x)

    x = RE_WS.sub(" ", x).strip()
    return x

def keep_example_clean(q_clean: str, a_clean: str, speciality: str = "") -> bool:
    if q_clean is None or a_clean is None:
        return False
    if not str(q_clean).strip() or not str(a_clean).strip():
        return False

    if speciality is None or not str(speciality).strip():
        return False

    if RE_ANY_STAR.search(q_clean) or RE_ANY_STAR.search(a_clean):
        return False

    if ("@" in q_clean) or ("@" in a_clean):
        return False
    if RE_MAIL_TOKEN.search(q_clean) or RE_MAIL_TOKEN.search(a_clean):
        return False

    if (RE_URL.search(q_clean) or RE_EMAIL.search(q_clean) or RE_PHONE.search(q_clean) or RE_PHONE_TR_LOOSE.search(q_clean)):
        return False
    if (RE_URL.search(a_clean) or RE_EMAIL.search(a_clean) or RE_PHONE.search(a_clean) or RE_PHONE_TR_LOOSE.search(a_clean)):
        return False

    if RE_RX_Q.search(q_clean):
        return False

    if RE_DOSAGE.search(a_clean) or RE_FREQ.search(a_clean):
        return False

    if len(q_clean) < 20 or len(q_clean) > 2000:
        return False
    if len(a_clean) < 50 or len(a_clean) > 2500:
        return False

    return True

def map_clean(ex: dict) -> dict:
    q_raw = ex.get("question") or ex.get("question_content") or ex.get("soru") or ""
    a_raw = ex.get("answer")   or ex.get("question_answer")   or ex.get("cevap") or ""
    spec  = ex.get("speciality") or ex.get("doctor_speciality") or ex.get("alan") or ""
    title = ex.get("title") or ex.get("doctor_title") or ex.get("unvan") or ""

    q_clean = clean_text(q_raw, is_answer=False)
    a_clean = clean_text(a_raw, is_answer=True)

    keep = keep_example_clean(q_clean, a_clean, speciality=spec)

    return {
        "doctor_title": title,
        "doctor_speciality": spec,
        "q_clean": q_clean,
        "a_clean": a_clean,
        "keep": keep,
    }


In [ ]:

train_c = (
    train_raw
    .map(map_clean, desc="Cleaning train")
    .filter(lambda x: x["keep"], desc="Filtering train")
)

test_c = None
if val_raw is not None:
    test_c = (
        val_raw
        .map(map_clean, desc="Cleaning test")
        .filter(lambda x: x["keep"], desc="Filtering test")
    )

print("After clean:", len(train_c), (len(test_c) if test_c is not None else None))


Cleaning train:   0%|          | 0/503196 [00:00<?, ? examples/s]

Filtering train:   0%|          | 0/503196 [00:00<?, ? examples/s]

Cleaning test:   0%|          | 0/60000 [00:00<?, ? examples/s]

Filtering test:   0%|          | 0/60000 [00:00<?, ? examples/s]

After clean: 327477 40690


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

TARGET_TOTAL = 60_000
NUM_CLASSES_TARGET = 20
VAL_RATIO = 0.10
SEED = 42

df = train_c.to_pandas()
df = df.dropna(subset=["doctor_speciality", "q_clean", "a_clean"]).copy()
df["doctor_speciality"] = df["doctor_speciality"].astype(str).str.strip()
df = df[df["doctor_speciality"] != ""].copy()

df = df.drop_duplicates(subset=["q_clean"]).copy()

vc = df["doctor_speciality"].value_counts()

num_classes = min(NUM_CLASSES_TARGET, len(vc))
while True:
    base = TARGET_TOTAL // num_classes
    eligible = vc[vc >= base].index.tolist()
    if len(eligible) >= num_classes:
        top_specs = eligible[:num_classes]
        break
    num_classes = len(eligible)
    if num_classes < 2:
        raise ValueError(
            f"Temizlik sonrası dengeli örnekleme için yeterli branş kalmadı. "
            f"Mevcut branş sayısı: {len(vc)}"
        )

base = TARGET_TOTAL // num_classes
rem  = TARGET_TOTAL % num_classes

parts = []
for i, spec in enumerate(top_specs):
    n = base + (1 if i < rem else 0)
    g = df[df["doctor_speciality"] == spec]
    n = min(len(g), n)
    parts.append(g.sample(n=n, random_state=SEED + i))

df_bal = pd.concat(parts, ignore_index=True)

print("Balanced size:", len(df_bal), "| target:", TARGET_TOTAL)
print("Num classes:", num_classes, "| base per class:", base, "| remainder:", rem)
print("Per-class min/max:",
      df_bal["doctor_speciality"].value_counts().min(),
      df_bal["doctor_speciality"].value_counts().max())

train_df, val_df = train_test_split(
    df_bal,
    test_size=VAL_RATIO,
    random_state=SEED,
    stratify=df_bal["doctor_speciality"]
)

print("Train/Val:", len(train_df), len(val_df))


Balanced size: 60000 | target: 60000
Num classes: 20 | base per class: 3000 | remainder: 0
Per-class min/max: 3000 3000
Train/Val: 54000 6000


In [ ]:
from datasets import Dataset
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

SYSTEM_MESSAGE = (
    "Sen bir sağlık bilgilendirme asistanısın; doktor değilsin. "
    "Kesin tanı/teşhis koyamazsın. İlaç adı, doz, reçete, tedavi planı öneremezsin. "
    "Kişisel veri, hastane/klinik adı, doktor adı, iletişim bilgisi veremezsin veya isteyemezsin. "
    "Genel, bilimsel ve tarafsız bilgi ver. "
    "Acil belirtilerde kullanıcıyı 112/acil servise yönlendir."
)

train_ds = Dataset.from_pandas(
    train_df[["doctor_speciality", "q_clean", "a_clean"]].copy(),
    preserve_index=False,
)
val_ds = Dataset.from_pandas(
    val_df[["doctor_speciality", "q_clean", "a_clean"]].copy(),
    preserve_index=False,
)

def to_text(ex):
    spec = (ex.get("doctor_speciality") or "").strip()
    q    = (ex.get("q_clean") or "").strip()
    a    = (ex.get("a_clean") or "").strip()

    user_msg = f"Branş: {spec}\nSoru: {q}"
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": user_msg},
        {"role": "assistant", "content": a},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

train_dataset = train_ds.map(to_text, remove_columns=train_ds.column_names)
val_dataset   = val_ds.map(to_text, remove_columns=val_ds.column_names)

print(train_dataset[0]["text"][:600])


Map:   0%|          | 0/54000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

Sen bir sağlık bilgilendirme asistanısın; doktor değilsin. Kesin tanı/teşhis koyamazsın. İlaç adı, doz, reçete, tedavi planı öneremezsin. Kişisel veri, hastane/klinik adı, doktor adı, iletişim bilgisi veremezsin veya isteyemezsin. Genel, bilimsel ve tarafsız bilgi ver. Acil belirtilerde kullanıcıyı 112/acil servise yönlendir.<|eot_id|><|start_header_id|>user<|end_header_id|>

Branş: dermatoloji
Soru: Prematüre 33 haftalık doğan bebeğim var, şu an 5 aylık (düzeltilmiş yaş


In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, pad_to_multiple_of=8),
    dataset_num_proc=4,
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=16,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        num_train_epochs=1,

        learning_rate=1e-4,
        warmup_ratio=0.03,
        lr_scheduler_type="cosine",
        optim="adamw_8bit",
        weight_decay=0.01,
        max_grad_norm=1.0,

        logging_steps=10,
        eval_strategy="steps",
        eval_steps=250,
        save_strategy="steps",
        save_steps=250,
        save_total_limit=2,

        seed=3407,
        output_dir="outputs",
        report_to="none",
        bf16=True,
        fp16=False,
        group_by_length=True,
    ),
)

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/54000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/6000 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/54000 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
7.137 GB of memory reserved.


In [ ]:
print("Training starting...")
trainer_stats = trainer.train()

Training starting...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 54,000 | Num Epochs = 1 | Total steps = 844
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
250,2.198600,2.273541
500,2.100200,2.164358


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Step,Training Loss,Validation Loss
250,2.198600,2.273541
500,2.100200,2.164358
750,2.100300,2.126187


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

5352.7307 seconds used for training.
89.21 minutes used for training.
Peak reserved memory = 15.709 GB.
Peak reserved memory for training = 8.572 GB.
Peak reserved memory % of max memory = 39.712 %.
Peak reserved memory for training % of max memory = 21.67 %.


In [ ]:
from unsloth import FastLanguageModel
import torch

FastLanguageModel.for_inference(model)
model.eval()

@torch.inference_mode()
def generate_answer(user_text, max_new_tokens=256):
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": user_text},
    ]

    enc = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )

    if isinstance(enc, torch.Tensor):
        input_ids = enc.to("cuda")
        attention_mask = torch.ones_like(input_ids, device="cuda")
        input_len = input_ids.shape[-1]
        gen_kwargs = {"input_ids": input_ids, "attention_mask": attention_mask}
    else:
        enc = {k: v.to("cuda") for k, v in enc.items()}
        input_len = enc["input_ids"].shape[-1]
        gen_kwargs = enc

    out = model.generate(
        **gen_kwargs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.1,
        use_cache=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    gen_tokens = out[0][input_len:]
    return tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

print(generate_answer("3 gündür boğazım ağrıyor, ateşim yok. Ne yapmalıyım?"))


Merhaba, boğaz ağrısı için antibiyotik kullanmanız gerekmeyebilir. Ağızdan alınabilecek ilaçlar var. Doktorunuza başvurun. Geçmiş olsun.


In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

In [ ]:
if True: model.push_to_hub_gguf("SemihBekdas/Llama3.1-8B-TR-PatientQA-LoRA-v1", tokenizer, quantization_method = "q4_k_m", token = "Token Girilecek")


Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Unsloth: Merge process complete. Saved to `/tmp/unsloth_gguf_slkyftbt`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['Meta-Llama-3.1-8B-Instruct.BF1

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...1-8B-Instruct.Q4_K_M.gguf:   0%|          |  559kB / 4.92GB            

Uploading config.json...
Uploading Ollama Modelfile...
Unsloth: Successfully uploaded GGUF to https://huggingface.co/SemihBekdas/Llama3.1-8B-TR-PatientQA-LoRA-v1
Unsloth: Cleaning up temporary files...
